# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [1]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

input_path = './Data/Antera_inputs.xlsx'
input_sheet = 'Antera_spray'
output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_Antera.xlsx'

iterations = 100000
chunks = '1000'  #'auto' or more than 500

In [2]:
from Tools_Antera import *
from Model_Antera import *

# LCI

The aircraft's life cycle was divided into four parts:
1. Development and Engineering
2. Manufacturing and Assembly
3. Operation
4. End-of-Life

First, we execute function to read inputs from designated `input_path` and transform it into Parameters and ParameterSet object `p`.

In [3]:
inputs = read_inputs(input_path, input_sheet)

In [4]:
p = ParameterSet(inputs, iterations, chunks)

Samples are stored in a xr.Dataset object accessible throught the `p.data` ParameterSet attribute.

Then, we run the function that calculates the functional unit for the study.

In [5]:
p = func_unit(p)

In [6]:
p.data["ha_life"].mean().compute()

<xarray.DataArray 'ha_life' ()>
array(408453.74673347)

In [7]:
p.data.load()

<xarray.Dataset>
Dimensions:             (i: 100000)
Coordinates:
  * i                   (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables: (12/91)
    takt_US             (i) float64 2.184 2.309 2.406 ... 2.252 2.468 2.496
    takt_BR             (i) float64 2.392 2.261 2.375 2.531 ... 2.55 2.171 2.261
    OEW                 (i) float64 1e+03 1e+03 1e+03 ... 1e+03 1e+03 1e+03
    flights_year        (i) float64 1.878e+03 1.844e+03 ... 1.821e+03 1.911e+03
    FH                  (i) float64 1.907 1.858 1.772 1.874 ... 1.628 1.926 1.53
    productivity        (i) float64 5.0 5.0 5.0 5.0 5.0 ... 5.0 5.0 5.0 5.0 5.0
    ...                  ...
    ha_flight           (i) float64 9.537 9.289 8.859 9.37 ... 8.138 9.628 7.649
    ha_year             (i) float64 1.791e+04 1.713e+04 ... 1.753e+04 1.462e+04
    ha_life             (i) float64 4.238e+05 4.509e+05 ... 4.019e+05 3.923e+05
    ha_fleet            (i) float64 3.39e+09 3.607e+09 ... 3.216e+09 3.138e+09
    ha_fleet_US         (i) float64 1.695e+09 1.804e+09 ... 1.608e+09 1.569e+09
    ha_fleet_BR         (i) float64 1.695e+09 1.804e+09 ... 1.608e+09 1.569e+09

<br></br>
Executing the UP reading and writing functions.

In [8]:
UP_dataframe = read_unit_processes(database_path)

In [9]:
UP = unit_process_dataset(UP_dataframe)

Building the empty LCI to start the calculation.

In [10]:
inventory = LCI(name=input_sheet, iterations=iterations, UP=UP, parameters=p)

In [11]:
inventory.run()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 100000)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
  * i               (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables: (12/16)
    Office          (Substances, i) float64 -9.117e-12 -1.039e-11 ... 2.207e-12
    Infrastructure  (Substances, i) float64 7.623e-13 7.254e-13 ... -3.26e-13
    Capital         (Substances, i) float64 2.84e-14 2.669e-14 ... -2.886e-12
    Materials       (Substances, i) float64 -1.79e-10 -1.721e-10 ... 5.314e-10
    Factory         (Substances, i) float64 -1.644e-10 -1.814e-10 ... 3.573e-11
    Logistics       (Substances, i) float64 1.274e-13 1.522e-13 ... 3.465e-12
    ...              ...
    Fuel            (Substances, i) float64 1.857e-10 1.958e-10 ... 1.868e-08
    Recycling       (Substances, i) float64 dask.array<chunksize=(1835, 6250), meta=np.ndarray>
    Incineration    (Substances, i) float64 dask.array<chunksize=(1835, 6250), meta=np.ndarray>
    Landfill        (Substances, i) float64 dask.array<chunksize=(1835, 6250), meta=np.ndarray>
    Prototypes      (Substances, i) float64 -5.67e-19 -5.485e-19 ... 1.018e-18
    Certification   (Substances, i) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    Name:     Antera_spray

In [12]:
inventory.data.load()

<xarray.Dataset>
Dimensions:         (Substances: 1835, i: 100000)
Coordinates:
    Units           (Substances) object 'kg' 'kg' 'kg' 'kg' ... 'kg' 'kBq' 'kBq'
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
  * i               (i) int32 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
Data variables: (12/16)
    Office          (Substances, i) float64 -9.117e-12 -1.039e-11 ... 2.207e-12
    Infrastructure  (Substances, i) float64 7.623e-13 7.254e-13 ... -3.26e-13
    Capital         (Substances, i) float64 2.84e-14 2.669e-14 ... -2.886e-12
    Materials       (Substances, i) float64 -1.79e-10 -1.721e-10 ... 5.314e-10
    Factory         (Substances, i) float64 -1.644e-10 -1.814e-10 ... 3.573e-11
    Logistics       (Substances, i) float64 1.274e-13 1.522e-13 ... 3.465e-12
    ...              ...
    Fuel            (Substances, i) float64 1.857e-10 1.958e-10 ... 1.868e-08
    Recycling       (Substances, i) float64 -2.124e-05 -1.996e-05 ... -8.389e-11
    Incineration    (Substances, i) float64 2.124e-05 1.996e-05 ... -3.93e-12
    Landfill        (Substances, i) float64 2.124e-05 1.996e-05 ... 9.665e-14
    Prototypes      (Substances, i) float64 -5.67e-19 -5.485e-19 ... 1.018e-18
    Certification   (Substances, i) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    Name:     Antera_spray

The LCI data can be accessed by the `LCI.data` attribute.

# LCIA

Extracting the characterization factors from the excel file located in `database_path`.

In [13]:
MP_data, EP_data = read_CF(database_path)

Creating an instance of the characterization factors class:

In [14]:
CF = CharactFactors(MP_data, EP_data, UP.Substances)

Adjusting the NLT category CFs.

In [15]:
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data)
CF.MP.to_array();

The impacts CFs can be accessed by the `CharactFactors.Midpoint.dataset` or `CharactFactors.Endpoint.dataset` attributes.

In [16]:
display(CF.MP.dataset)
display(CF.EP.dataset)

<xarray.Dataset>
Dimensions:         (Substances: 1835)
Coordinates:
  * Substances      (Substances) MultiIndex
  - name            (Substances) object '1,4-Butanediol' ... 'Zirconium-95'
  - compartment     (Substances) object 'Water' 'Air' 'Air' ... 'Water' 'Air'
  - subcompartment  (Substances) object 'surface water' ... 'non-urban air or...
Data variables: (12/18)
    FE              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MRD             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    POF             (Substances) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    MET             (Substances) float64 0.0 0.0 0.000283 ... 0.0 0.0 0.0
    TA              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ULO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...              ...
    ALO             (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    HT              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    IR              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    FD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    WD              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    CC              (Substances) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

<xarray.Dataset>
Dimensions:     (Categories: 18)
Coordinates:
  * Categories  (Categories) object 'CC' 'OD' 'TA' 'FE' ... 'WD' 'MRD' 'FD'
Data variables:
    HH          (Categories) float64 1.4e-06 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ED          (Categories) float64 7.93e-09 0.0 5.8e-09 ... 0.0 0.0 0.0
    RA          (Categories) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0715 0.165

<br></br>
**Executing the Impact Assessment.**

In [17]:
aeco = LCIA.build(inventory, CF)

MemoryError: Unable to allocate 24.6 GiB for an array with shape (1835, 100000, 18) and data type float64

## Midpoint Results

Raw midpoint results can be accessed by the `LCIA.MP` attribute. Mean midpoint results are calculated by the `LCIA.mean` method, with pathway set to "MP".

In [ ]:
display(aeco.MP)

<xarray.Dataset>
Dimensions:         (Categories: 18, i: 1000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * Categories      (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'FD' 'WD' 'CC'
Data variables: (12/16)
    Office          (i, Categories) float64 1.036e-06 0.0006857 ... 0.04059
    Infrastructure  (i, Categories) float64 3.366e-06 0.02048 ... 0.01204
    Capital         (i, Categories) float64 9.281e-07 0.0004122 ... 0.001071
    Materials       (i, Categories) float64 2.911e-05 0.02332 ... 0.07448
    Factory         (i, Categories) float64 2.198e-05 0.0144 ... 0.01186 0.9976
    Logistics       (i, Categories) float64 2.728e-06 0.0006472 ... 0.04454
    ...              ...
    Fuel            (i, Categories) float64 0.01599 2.094 ... 0.2886 87.42
    Recycling       (i, Categories) float64 -0.0001822 -6.179 ... -4.29
    Incineration    (i, Categories) float64 0.0001744 6.177 ... 0.0001004 4.281
    Landfill        (i, Categories) float64 0.0001747 6.177 ... 9.983e-05 4.282
    Prototypes      (i, Categories) float64 9.989e-14 7.123e-11 ... 1.733e-09
    Certification   (i, Categories) float64 0.0 0.0 ... 0.000554 0.001419
Attributes:
    Name:     Antera_spray

In [ ]:
aeco.mean("MP", by='sum').compute()

<xarray.DataArray (Categories: 18)>
array([2.60285501e-02, 1.03774006e+01, 1.96220660e+00, 7.42469026e+00,
       1.58087872e+00, 8.46152060e-01, 5.79497317e-01, 5.28078831e-01,
       2.95107160e+00, 3.93297005e-03, 1.01501439e-03, 8.28232630e-01,
       8.58307977e+00, 1.05072694e+04, 4.53740671e+01, 2.59857876e+02,
       2.72081513e+02, 7.71109408e+02])
Coordinates:
  * Categories  (Categories) <U3 'FE' 'MRD' 'POF' 'MET' ... 'IR' 'FD' 'WD' 'CC'

## Endpoint Results

Raw endpoint results can be accessed by the `LCIA.EP` attribute. Mean endpoint results are calculated by the `LCIA.mean` method, with pathway set to "EP".

In [46]:
display(aeco.EP)

<xarray.Dataset>
Dimensions:         (AOP: 3, i: 1000)
Coordinates:
  * i               (i) int32 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * AOP             (AOP) <U2 'HH' 'ED' 'RA'
Data variables: (12/16)
    Office          (i, AOP) float64 6.648e-08 3.507e-10 ... 3.227e-10 0.002788
    Infrastructure  (i, AOP) float64 3.574e-08 1.149e-10 ... 9.651e-11 0.001916
    Capital         (i, AOP) float64 4.042e-09 1.035e-11 ... 8.654e-12 7.686e-05
    Materials       (i, AOP) float64 2.556e-07 7.359e-10 ... 6.049e-10 0.005794
    Factory         (i, AOP) float64 1.494e-06 7.893e-09 ... 7.93e-09 0.06849
    Logistics       (i, AOP) float64 1.025e-07 4.299e-10 ... 3.555e-10 0.002612
    ...              ...
    Fuel            (i, AOP) float64 0.0002162 7.569e-07 ... 7.018e-07 40.49
    Recycling       (i, AOP) float64 -0.007784 -1.21e-07 ... -1.011e-07 -0.3698
    Incineration    (i, AOP) float64 0.007784 1.209e-07 ... 1.011e-07 0.3692
    Landfill        (i, AOP) float64 0.007784 1.209e-07 ... 1.011e-07 0.3693
    Prototypes      (i, AOP) float64 3.438e-15 1.682e-17 ... 1.38e-17 1.193e-10
    Certification   (i, AOP) float64 3.081e-09 1.615e-11 0.0 ... 1.126e-11 0.0
Attributes:
    Name:     Antera_spray

In [47]:
aeco.mean("EP", by="sum").compute()

<xarray.DataArray (AOP: 3)>
array([8.58613168e-03, 6.20880989e-06, 4.36185337e+01])
Coordinates:
  * AOP      (AOP) <U2 'HH' 'ED' 'RA'

## Contribution to Variance (CTV)

Calculate CTV results for all inputs with midpoint and endpoint results. CTV results can be acessed by the `LCIA.CTV` attribute.

In [48]:
aeco.build_CTV(parameterset=p)

<xarray.Dataset>
Dimensions:     (Parameters: 84)
Coordinates:
  * Parameters  (Parameters) <U18 'takt_US' 'takt_BR' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    MRD         (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    POF         (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    MET         (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    TA          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    ULO         (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    ...          ...
    FD          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    WD          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    CC          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    HH          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    ED          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>
    RA          (Parameters) float64 dask.array<chunksize=(84,), meta=np.ndarray>

In [49]:
aeco.CTV.load()

c:\Users\giparoli\Documents\Projetos\AEco\Scripts\Tools_Antera.py:309: RuntimeWarning: invalid value encountered in true_divide
  return covariance_gufunc(x, y) / (x.std(axis=-1) * y.std(axis=-1))


<xarray.Dataset>
Dimensions:     (Parameters: 84)
Coordinates:
  * Parameters  (Parameters) <U18 'takt_US' 'takt_BR' ... 'p_recycl_CFRP'
Data variables: (12/21)
    FE          (Parameters) float64 0.02222 0.002931 0.0 ... 0.0 0.0 0.0
    MRD         (Parameters) float64 0.0313 0.04507 0.0 2.704 ... 0.0 0.0 0.0
    POF         (Parameters) float64 0.04116 0.1082 0.0 0.0243 ... 0.0 0.0 0.0
    MET         (Parameters) float64 0.09443 0.009022 0.0 ... 0.0 0.0 0.0
    TA          (Parameters) float64 0.03795 0.09158 0.0 0.0283 ... 0.0 0.0 0.0
    ULO         (Parameters) float64 0.02976 0.07275 0.0 0.02436 ... 0.0 0.0 0.0
    ...          ...
    FD          (Parameters) float64 0.04265 0.1146 0.0 0.02493 ... 0.0 0.0 0.0
    WD          (Parameters) float64 0.03949 0.09522 0.0 0.02759 ... 0.0 0.0 0.0
    CC          (Parameters) float64 0.04298 0.1149 0.0 0.02006 ... 0.0 0.0 0.0
    HH          (Parameters) float64 0.002447 0.1957 0.0 6.089 ... 0.0 0.0 0.0
    ED          (Parameters) float64 0.04313 0.1182 0.0 0.01176 ... 0.0 0.0 0.0
    RA          (Parameters) float64 0.04179 0.114 0.0 0.01739 ... 0.0 0.0 0.0

In [55]:
aeco.CTV["CC"].sortby(aeco.CTV["CC"], ascending=False)

<xarray.DataArray 'CC' (Parameters: 84)>
array([3.35067012e+01, 3.13252661e+01, 3.03328897e+01, 1.16417520e+00,
       4.89664686e-01, 3.56410321e-01, 2.66249770e-01, 1.80995267e-01,
       1.73651719e-01, 1.64748054e-01, 1.61977565e-01, 1.54803219e-01,
       1.42555202e-01, 1.35062552e-01, 1.27950918e-01, 1.26625524e-01,
       1.14942448e-01, 1.10426995e-01, 8.66316627e-02, 8.33877449e-02,
       8.18529710e-02, 7.03762736e-02, 6.88946117e-02, 5.62147775e-02,
       4.29817092e-02, 4.27847157e-02, 3.59729661e-02, 3.37132661e-02,
       3.33715237e-02, 3.25282136e-02, 3.09736592e-02, 2.55904399e-02,
       2.49364528e-02, 2.22213963e-02, 2.10628426e-02, 2.10370923e-02,
       2.00608655e-02, 1.75936242e-02, 1.74519634e-02, 1.54707673e-02,
       1.25502837e-02, 1.18342621e-02, 1.02165037e-02, 8.55089803e-03,
       5.77522805e-03, 4.96480503e-03, 4.89450464e-03, 4.57679601e-03,
       4.40871547e-03, 4.35101511e-03, 2.96589497e-03, 1.96813864e-03,
       1.36533517e-03, 8.64774784e-04, 3.05777908e-04, 2.00980478e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])
Coordinates:
  * Parameters  (Parameters) <U18 'FH' 't_cruise' ... 'productivity' 'OEW'

# Export Results

Export MP, EP, LCI and CTV result to NetCDF file.

In [56]:
aeco.save(output_path, LCI=False)

In [22]:
#aeco.to_excel(output_path)